# BigQuery - BQML Online Predictions

This notebook exports the BigQuery ML model created in `02 - BigQuery - BQML` and then uses Vertex AI to upload the model and deploy it to an endpoint for online predictions.

**Prerequisites**
- `00 - Initial Setup`
- `01 - BigQuery - Data`
- `02 - BigQuery - BQML`

**Overview**

<img src="architectures/statmike-mlops-03.png">

---
## Setup

Setup Parameters For Model Deployment

In [2]:
PROJECT_ID='statmike-mlops'
REGION='us-central1'

MODEL_NAME='MODEL_03_BQML-DIGITS'
ENDPOINT_NAME='ENDPOINT_03_BQML-DIGITS'
MODEL_DIR='gs://{}/digits/model/03_bqml'.format(PROJECT_ID)
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

params = {"MODEL_DIR":MODEL_DIR}
DEPLOY_IMAGE='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-2:latest'
DEPLOY_COMPUTE='n1-standard-4'

Setup AI Platform Python Clients
- https://googleapis.dev/python/aiplatform/latest/index.html

In [15]:
from google.cloud import aiplatform

from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

aiplatform.init(project=PROJECT_ID, location=REGION)

---
## Export the BigQuery Model

Export the BigQuery Model:
- https://cloud.google.com/bigquery-ml/docs/exporting-models

In [4]:
%%bigquery --params $params
EXPORT MODEL `digits.digits_lr`
OPTIONS(URI = @MODEL_DIR)

Query complete after 0.01s: 100%|██████████| 2/2 [00:00<00:00, 1074.50query/s]                        


""


---
## Serving

### Upload The Model

In [5]:
model = aiplatform.Model.upload(
    display_name = MODEL_NAME,
    serving_container_image_uri = DEPLOY_IMAGE,
    artifact_uri = MODEL_DIR
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/691911073727/locations/us-central1/models/7568079271470563328/operations/1520511056638640128
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/691911073727/locations/us-central1/models/7568079271470563328
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/691911073727/locations/us-central1/models/7568079271470563328')


In [6]:
model.display_name

'MODEL_03_BQML-DIGITS'

### Create An Endpoint

In [7]:
endpoint = aiplatform.Endpoint.create(display_name = ENDPOINT_NAME)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/691911073727/locations/us-central1/endpoints/8864051636897579008/operations/750395520358285312
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/691911073727/locations/us-central1/endpoints/8864051636897579008
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/691911073727/locations/us-central1/endpoints/8864051636897579008')


In [8]:
endpoint.display_name

'ENDPOINT_03_BQML-DIGITS'

### Deploy Model To Endpoint

In [9]:
endpoint.deploy(
    model=model,
    deployed_model_display_name=MODEL_NAME+'_DEPLOYED',
    traffic_percentage = 100,
    machine_type = 'n1-standard-4',
    min_replica_count = 1,
    max_replica_count = 1
)

INFO:google.cloud.aiplatform.models:Deploying Model projects/691911073727/locations/us-central1/models/7568079271470563328 to Endpoint : projects/691911073727/locations/us-central1/endpoints/8864051636897579008
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/691911073727/locations/us-central1/endpoints/8864051636897579008/operations/8861378449252548608
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/691911073727/locations/us-central1/endpoints/8864051636897579008


---
## Prediction

### Data For Prediction

In [10]:
%%bigquery pred
SELECT *
FROM `digits.digits_source`
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:01<00:00,  8.77rows/s]


In [11]:
pred

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p56,p57,p58,p59,p60,p61,p62,p63,target,target_OE
0,0.0,5.0,16.0,15.0,5.0,0.0,0.0,0.0,0.0,2.0,...,0.0,6.0,16.0,16.0,16.0,16.0,7.0,0.0,2,Even
1,0.0,5.0,16.0,12.0,1.0,0.0,0.0,0.0,0.0,5.0,...,0.0,8.0,16.0,16.0,16.0,16.0,4.0,0.0,2,Even
2,0.0,5.0,15.0,16.0,6.0,0.0,0.0,0.0,0.0,11.0,...,0.0,6.0,16.0,16.0,16.0,13.0,3.0,0.0,2,Even
3,0.0,4.0,15.0,15.0,8.0,0.0,0.0,0.0,0.0,8.0,...,0.0,7.0,14.0,11.0,0.0,0.0,0.0,0.0,2,Even
4,0.0,6.0,16.0,16.0,16.0,15.0,10.0,0.0,0.0,9.0,...,0.0,9.0,16.0,11.0,0.0,0.0,0.0,0.0,5,Odd
5,0.0,8.0,16.0,12.0,15.0,16.0,7.0,0.0,0.0,13.0,...,0.0,7.0,16.0,16.0,10.0,0.0,0.0,0.0,5,Odd
6,0.0,8.0,13.0,15.0,16.0,16.0,8.0,0.0,0.0,9.0,...,0.0,9.0,16.0,6.0,0.0,0.0,0.0,0.0,5,Odd
7,0.0,7.0,12.0,14.0,16.0,8.0,0.0,0.0,0.0,8.0,...,0.0,9.0,12.0,0.0,0.0,0.0,0.0,0.0,7,Odd
8,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0,Even
9,0.0,0.0,1.0,9.0,15.0,11.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,10.0,13.0,3.0,0.0,0.0,0,Even


### Prepare Prediction Request

In [12]:
pred.loc[:0]

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p56,p57,p58,p59,p60,p61,p62,p63,target,target_OE
0,0.0,5.0,16.0,15.0,5.0,0.0,0.0,0.0,0.0,2.0,...,0.0,6.0,16.0,16.0,16.0,16.0,7.0,0.0,2,Even


In [13]:
newob = pred.loc[:0,'p0':'p63'].to_dict(orient='records')[0]

In [16]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Prediction

In [17]:
prediction = endpoint.predict(instances=instances,parameters=parameters)

In [18]:
prediction

Prediction(predictions=[{'target_probs': [0.0004684903316369283, 2.794953040795491e-05, 1.639273465457618e-05, 0.001516294851701739, 0.0002815857793980461, 9.305114984565324e-06, 0.0003273592649797026, 0.9967314480731848, 5.287317575313678e-06, 0.0006158870014763913], 'target_values': ['6', '0', '7', '5', '3', '4', '1', '2', '9', '8'], 'predicted_target': ['2']}], deployed_model_id='3576852062643683328', explanations=None)

In [19]:
prediction.predictions[0]

{'target_probs': [0.0004684903316369283,
  2.794953040795491e-05,
  1.639273465457618e-05,
  0.001516294851701739,
  0.0002815857793980461,
  9.305114984565324e-06,
  0.0003273592649797026,
  0.9967314480731848,
  5.287317575313678e-06,
  0.0006158870014763913],
 'target_values': ['6', '0', '7', '5', '3', '4', '1', '2', '9', '8'],
 'predicted_target': ['2']}

In [21]:
import numpy as np

prediction.predictions[0]['target_values'][np.argmax(prediction.predictions[0]['target_probs'])]

'2'

---
## Remove Resources
see notebook "XX - Cleanup"